In [1]:
!pip install torch ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 977.1/977.1 kB 86.9 MB/s eta 0:00:00


In [2]:
"""
Importing modules

"""

import os
import warnings

warnings.filterwarnings("ignore")

import shutil
import matplotlib.pyplot as plt
import yaml
import torch
import ultralytics
from tqdm.auto import tqdm
from ultralytics import YOLO

# Set base directory to the current working directory
base_dir = os.getcwd()  # Automatically gets the current directory
print(f"Using base directory: {base_dir}")

# Define dataset paths
train_source_images_dir = os.path.join(base_dir, "images_cassette_tong_dan/train")
train_source_labels_dir = os.path.join(base_dir, "labels_cassette_tong_dan/train")
val_source_images_dir = os.path.join(base_dir, "images_cassette_tong_dan/val")
val_source_labels_dir = os.path.join(base_dir, "labels_cassette_tong_dan/val")

# Set local working directory for training
train_destination_dir = os.path.join(base_dir, "train_cassette_tong_dan")
val_destination_dir = os.path.join(base_dir, "val_cassette_tong_dan")

# Create directories if they don't exist
os.makedirs(train_destination_dir, exist_ok=True)
os.makedirs(val_destination_dir, exist_ok=True)


# Function to copy files from source to destination
def copy_files(source_dir, destination_dir):
    """
    Move the files to their correct directories

    Args:
        source_dir (str): the source directory, where the files currently are
        destination_dir (): the target directory
    """
    if not os.path.exists(source_dir):
        print(f"Warning: Source directory {source_dir} does not exist. Skipping.")
        return
    for filename in tqdm(os.listdir(source_dir), desc=f"Copying from {source_dir}"):
        src_path = os.path.join(source_dir, filename)
        dst_path = os.path.join(destination_dir, filename)
        shutil.copy(src_path, dst_path)


# Copy train images and labels
copy_files(train_source_images_dir, train_destination_dir)
copy_files(train_source_labels_dir, train_destination_dir)

# Copy val images and labels
copy_files(val_source_images_dir, val_destination_dir)
copy_files(val_source_labels_dir, val_destination_dir)


# Validate file consistency (Image-Label pairs check)
def validate_files(directory):
    """
    Ensure that each image has a matching label file

    Args:
        directory (str): the directory where the images and labels are
    """
    files = os.listdir(directory)
    image_extensions = {".jpg", ".jpeg", ".png"}  # Adjust based on dataset
    label_extension = ".txt"

    image_files = {
        os.path.splitext(f)[0]
        for f in files
        if os.path.splitext(f)[1] in image_extensions
    }
    label_files = {
        os.path.splitext(f)[0]
        for f in files
        if os.path.splitext(f)[1] == label_extension
    }

    unmatched_images = image_files - label_files
    unmatched_labels = label_files - image_files

    for img in unmatched_images:
        print(f"Image file {img} has no matching label.")
    for lbl in unmatched_labels:
        print(f"Label file {lbl} has no matching image.")

    print(f"Total matched pairs in {directory}: {len(image_files & label_files)}")


validate_files(train_destination_dir)
validate_files(val_destination_dir)

# Create dataset.yaml configuration
dataset_config = {
    "nc": 1,  # Number of classes (make sure it matches the number of labels in your dataset)
    "names": ["casset"],  # Class names
    "train": train_destination_dir,
    "val": val_destination_dir,
    "device": 0,
}

# Save YAML file
yaml_path = os.path.join(base_dir, "dataset_casset_tong_dan.yaml")
with open(yaml_path, "w") as file:
    yaml.dump(dataset_config, file, default_flow_style=False)

# Check Ultralytics setup
print(ultralytics.checks())

# Load YOLO model
# model = YOLO("yolov11n.pt")
model = YOLO("../source_model/yolov8m.pt")

# # Enable Automatic Mixed Precision for better performance
# if torch.cuda.is_available():
#     model.half()  # Converts model to half precision

# Move model to GPU if available
if torch.cuda.is_available():
    model.to(device=0)

# Train the model using the local dataset
results = model.train(
    data=yaml_path,
    batch=32,
    epochs=120,
    imgsz=640,
)

# Plot training results
result_img_path = os.path.join(base_dir, "runs/detect/train/results.png")
if os.path.exists(result_img_path):
    result_img = plt.imread(result_img_path)
    plt.figure(figsize=(10, 10))
    plt.imshow(result_img)
    plt.axis("off")
    plt.show()
else:
    print("Training results image not found!")


Ultralytics 8.3.100 🚀 Python-3.10.14 torch-2.2.2 CUDA:0 (NVIDIA L4, 22478MiB)
Setup complete ✅ (64 CPUs, 242.5 GB RAM, 78.8/135.0 GB disk)
None


100%|██████████| 49.7M/49.7M [00:00<00:00, 181MB/s] 


engine/trainer: task=detect, mode=train, model=../source_model/yolov8m.pt, data=/home/ec2-user/SageMaker/dungnm/custom_model/small_cassette_tong_dan/dataset_casset_tong_dan.yaml, epochs=120, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sh

100%|██████████| 755k/755k [00:00<00:00, 45.1MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     


...

                   all          9          4      0.386       0.75      0.393      0.258



120 epochs completed in 0.148 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train/weights/best.pt, 52.0MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.100 🚀 Python-3.10.14 torch-2.2.2 CUDA:0 (NVIDIA L4, 22478MiB)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 26.88it/s]


                   all          9          4      0.839       0.75      0.856      0.534
Speed: 0.1ms preprocess, 2.1ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train


<Figure size 1000x1000 with 1 Axes>